In [1]:
import os
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\noise_reduction_autoencoder\\research'

In [2]:
os.chdir('../')
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\noise_reduction_autoencoder'

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model('artifacts/models/autoencoder_final.h5')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 1)    │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 185,219 (723.52 KB)

 Trainable params: 185,217 (723.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = ''
os.environ['MLFLOW_TRACKING_USERNAME'] = ''
os.environ['MLFLOW_TRACKING_PASSWORD'] = ''

In [7]:
#update entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    test_data: dict  # Preprocessed test dataset
    all_params: dict
    mlflow_uri: str
    params_image_size: int
    params_batch_size: int

In [20]:
#update config manager
from src import logger
from src.utils.common import read_yaml, create_directories
from src.entity.config_entity import DataPreprocessingConfig, ModelTrainingConfig
from src.config.configuration import ConfigurationManager
from pathlib import Path
from src.components.data_preprocessing import DataPreprocessing

class ConfigurationManager:
    def __init__(self, config_filepath="config/config.yaml", params_filepath="params.yaml"):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        # Load the preprocessed test dataset
        data_preprocessing_config = Path(self.get_data_preprocessing_config())
        data_preprocessing = DataPreprocessing(config=data_preprocessing_config)
        datasets = data_preprocessing.preprocess()

        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/models/autoencoder_final.h5"),  # Updated to .h5
            test_data=datasets["test"],
            mlflow_uri="https://dagshub.com/PrarthanaGanegama/noise_reduction_autoencoder.mlflow",
            all_params=self.params,
            params_image_size=self.config.data_preprocessing.image_size,
            params_batch_size=self.config.data_preprocessing.batch_size
        )
        return eval_config

In [21]:
#update components
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from src import logger
#from src.entity.config_entity import EvaluationConfig
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np
import matplotlib.pyplot as plt
from src.utils.common import save_json

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        """Evaluate the model on the test set and compute metrics."""
        self.model = self.load_model(self.config.path_of_model)
        
        # Evaluate on test set (compute test MSE)
        self.score = self.model.evaluate(self.config.test_data)
        logger.info(f"Test loss (MSE): {self.score}")
        
        # Get a batch from the test set for visualization and additional metrics
        test_batch = next(iter(self.config.test_data.take(1)))
        noisy_images, clean_images = test_batch[0], test_batch[1]
        
        # Predict denoised images
        predicted_images = self.model.predict(noisy_images)
        
        # Compute PSNR and SSIM
        psnr_values = []
        ssim_values = []
        for i in range(len(clean_images)):
            clean_img = clean_images[i].numpy().squeeze()
            pred_img = predicted_images[i].squeeze()
            psnr_val = psnr(clean_img, pred_img, data_range=1.0)
            ssim_val = ssim(clean_img, pred_img, data_range=1.0)
            psnr_values.append(psnr_val)
            ssim_values.append(ssim_val)
        
        self.avg_psnr = np.mean(psnr_values)
        self.avg_ssim = np.mean(ssim_values)
        logger.info(f"Average PSNR: {self.avg_psnr:.2f} dB")
        logger.info(f"Average SSIM: {self.avg_ssim:.4f}")
        
        # Plot a few examples
        num_examples = 3
        plt.figure(figsize=(12, 4 * num_examples))
        for i in range(num_examples):
            plt.subplot(num_examples, 3, i * 3 + 1)
            plt.imshow(noisy_images[i], cmap="gray")
            plt.title("Noisy Image")
            plt.axis("off")
            
            plt.subplot(num_examples, 3, i * 3 + 2)
            plt.imshow(predicted_images[i], cmap="gray")
            plt.title("Denoised Image")
            plt.axis("off")
            
            plt.subplot(num_examples, 3, i * 3 + 3)
            plt.imshow(clean_images[i], cmap="gray")
            plt.title("Clean Image")
            plt.axis("off")
        
        plt.tight_layout()
        plot_path = Path("artifacts/plots/denoising_results.png")
        plt.savefig(plot_path)
        plt.close()
        logger.info(f"Denoising results saved to {plot_path}")
        
        self.save_score()

    def save_score(self):
        """Save evaluation metrics to a JSON file."""
        scores = {
            "test_mse": float(self.score),  # Convert to float for JSON serialization
            "average_psnr": float(self.avg_psnr),
            "average_ssim": float(self.avg_ssim)
        }
        save_json(path=Path("scores.json"), data=scores)
        logger.info("Evaluation scores saved to scores.json")

    def log_into_mlflow(self):
        """Log metrics, parameters, and artifacts to MLflow."""
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "test_mse": self.score,
                "average_psnr": self.avg_psnr,
                "average_ssim": self.avg_ssim
            })
            mlflow.log_artifact("artifacts/plots/denoising_results.png")
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="AutoencoderModel")
            else:
                mlflow.keras.log_model(self.model, "model")
        logger.info("Logged evaluation results to MLflow")

In [22]:
#update pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e

ERROR:NoiseReductionAutoencoder:'ConfigurationManager' object has no attribute 'get_data_preprocessing_config'
Traceback (most recent call last):
  File "C:\Users\Prarthana\AppData\Local\Temp\ipykernel_23352\2735673080.py", line 4, in <module>
    eval_config = config.get_evaluation_config()
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Prarthana\AppData\Local\Temp\ipykernel_23352\3367006845.py", line 17, in get_evaluation_config
    data_preprocessing_config = Path(self.get_data_preprocessing_config())
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'ConfigurationManager' object has no attribute 'get_data_preprocessing_config'


AttributeError: 'ConfigurationManager' object has no attribute 'get_data_preprocessing_config'